# Boltzmann's Machine

This notebook was created by Camille-Amaury JUGE, in order to better understand Boltzmann's Machine (BM) principles and how they work.

(it follows the exercices proposed by Hadelin de Ponteves on Udemy : https://www.udemy.com/course/le-deep-learning-de-a-a-z/)

## Imports

In [1]:
import numpy as np
import pandas as pd
# pytorch
import torch
import torch.nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

import sys
import csv

## Preprocessing

We are going to create a recommendation system based on a film's dataset.

It will try to predict if the user will love a film or not.

In [2]:
df_movies = pd.read_csv("ml-1m\\movies.dat", sep="::", header=None, engine="python",
                encoding="latin-1")
users = pd.read_csv("ml-1m\\users.dat", sep="::", header=None, engine="python",
                encoding="latin-1")
ratings = pd.read_csv("ml-1m\\ratings.dat", sep="::", header=None, engine="python",
                encoding="latin-1")

In [3]:
df_movies.head()
# movie id / name / type

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()
# user id / gender / age /professional category / Postal code

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()
# user id / movie id / rating / timestamp

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
df_train = pd.read_csv("ml-100k/u1.base", delimiter="\t", header=None)
df_test = pd.read_csv("ml-100k/u1.test", delimiter="\t", header=None)

In [7]:
df_train.head()
# user id / movie id / note / timestamp

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [8]:
df_test.head()
# user id / movie id / note / timestamp

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [9]:
#df_train = np.array(df_train, dtype="int")
#df_test = np.array(df_test, dtype="int")

Firstly, we will get the user's id set (as an array)

In [10]:
_users = list(set(np.concatenate((df_train[df_train.columns[0]].value_counts().index, 
                           df_test[df_test.columns[0]].value_counts().index), 
                          axis=0)))

In [11]:
print("Number of unique users : {}".format(len(_users)))

Number of unique users : 943


Nextly, we will get the movies's number

In [12]:
_movies =  list(set(np.concatenate((df_train[df_train.columns[1]].value_counts().index, 
                           df_test[df_test.columns[1]].value_counts().index), 
                          axis=0)))

In [13]:
print("Number of unique movies : {}".format(len(_movies)))

Number of unique movies : 1682


Then, we are going to convert our data to a huge matrix using pytorch

In [14]:
def createMatrix(df, users, movies):
    matrix = []
    movies_nb = len(movies)
    user_nb = len(users)
    df_array = np.array(df, dtype="int")
    for i,user in enumerate(users):
        filtered_movies = df_array[df_array[:,0] == user, 1]
        filtered_ratings = df_array[df_array[:,0] == user, 2]
        ratings = np.zeros(movies_nb)
        for j in range(len(filtered_movies)):
            ratings[filtered_movies[j] - 1] = filtered_ratings[j]
        matrix.append(ratings)
                      
        sys.stdout.write("\r Loading State : {} / {}".format(i+1,user_nb))
        sys.stdout.flush()
        
    return matrix

In [15]:
matrix = createMatrix(df_train, _users, _movies)

 Loading State : 943 / 943

In [16]:
matrix_test = createMatrix(df_test, _users, _movies)

 Loading State : 943 / 943

In [17]:
train = torch.FloatTensor(matrix)
test = torch.FloatTensor(matrix_test) 

We will now encode our ratings to -1 (no data = 0), 0 (don't love), 1 (love)

In [18]:
train[train == 0] = -1
test[test == 0] = -1

In [19]:
train[train < 3] = 0
test[test < 3] = 0

In [20]:
train[train >= 3] = 1
test[test >= 3] = 1

## Model

In [83]:
class RestrictedBoltzmannMachine():
    def __init__(self, nb_n_visible, nb_n_hidden):
        self.Weights = torch.randn(nb_n_visible, nb_n_hidden)
        self.a = torch.randn(1, nb_n_hidden)
        self.b = torch.randn(1, nb_n_visible)
    
    def gibs_sample_hidden(self, x):
        x_weights = torch.mm(x, self.Weights)
        probability_h_given_v = torch.sigmoid(x_weights + self.a.expand_as(x_weights))
        return probability_h_given_v, torch.bernoulli(probability_h_given_v)
    
    def gibs_sample_visible(self, y):
        y_weights = torch.mm(y, self.Weights.t())
        probability_v_given_h = torch.sigmoid(y_weights + self.b.expand_as(y_weights))
        return probability_v_given_h, torch.bernoulli(probability_v_given_h)
    
    def contrastive_divergence(self, visible_0, visible_k, 
                               probability_h_given_v0, probability_h_given_vk):
        self.Weights += torch.mm(visible_0.t(), probability_h_given_v0) - torch.mm(visible_k.t(), probability_h_given_vk)
        self.b += torch.sum((visible_0 - visible_k),0)
        self.a += torch.sum((probability_h_given_v0 - probability_h_given_vk),0)
        
    def train(self, X, epoch=10, batch_size=32, gibs_sampling=10):
        self.X_train = X
        nb_users = X.shape[0]
        for i in range(epoch):
            print("Epoch => {}/{}".format(i+1,epoch))
            train_loss = 0
            s = 0.
            for user in range(0, nb_users - batch_size, batch_size):
                visible_0 = X[user:user+batch_size]
                visible_k = visible_0
                probabilty_hidden_0, _ = self.gibs_sample_hidden(visible_0)
                for k in range(0,gibs_sampling):
                    _, hidden_k = self.gibs_sample_hidden(visible_k)
                    _, visible_k = self.gibs_sample_visible(hidden_k)
                    # no rating's filter --> no changes
                    visible_k[visible_0 < 0] = visible_0[visible_0 < 0]
                probabilty_hidden_k, _ = self.gibs_sample_hidden(visible_k)
                self.contrastive_divergence(visible_0, visible_k,
                                           probabilty_hidden_0, probabilty_hidden_k)
                train_loss += torch.mean(torch.abs(visible_0[visible_0 >= 0] - 
                                                   visible_k[visible_0 >= 0]))
                s+=1
            print("   => Loss : {}".format((train_loss/s)))
            
    def test(self, X):
        nb_users = self.X_train.shape[0]
        
        sys.stdout.write("\r Processing")
        sys.stdout.flush()
                         
        test_loss = 0
        s = 0.
        for user in range(nb_users):
            visible = self.X_train[user:user+1]
            visible_target = X[user:user+1]
            if len(visible_target[visible_target >= 0]) > 0:
                _, hidden = self.gibs_sample_hidden(visible)
                _, visible = self.gibs_sample_visible(hidden)
                test_loss += torch.mean(torch.abs(visible_target[visible_target >= 0] - 
                                                   visible[visible_target >= 0]))
                s+=1
                         
        sys.stdout.write("\r Test Set => Loss : {}".format((test_loss/s)))
        sys.stdout.flush()

In [84]:
_neuron_visible = train.shape[1]
_neuron_hidden = 100 #train.shape[0]
_batch_size = 1
_nb_epoch = 10
_gibs_sampling = 10

In [85]:
rbm = RestrictedBoltzmannMachine(_neuron_visible, _neuron_hidden)

In [86]:
rbm.train(train, _nb_epoch, _batch_size, _gibs_sampling)

Epoch => 1/10
   => Loss : 0.07236102223396301
Epoch => 2/10
   => Loss : 0.06743250787258148
Epoch => 3/10
   => Loss : 0.06691240519285202
Epoch => 4/10
   => Loss : 0.06655645370483398
Epoch => 5/10
   => Loss : 0.06641065329313278
Epoch => 6/10
   => Loss : 0.06626234948635101
Epoch => 7/10
   => Loss : 0.06591334939002991
Epoch => 8/10
   => Loss : 0.06564825028181076
Epoch => 9/10
   => Loss : 0.0654311254620552
Epoch => 10/10
   => Loss : 0.06551258265972137


In [87]:
rbm.test(test)

 Test Set => Loss : 0.04984850063920021